# EPL Dataset Analysis

## Load Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Load Data and Drop Columns

In [2]:
data_folder = '../data/raw/'
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# Dictionary to hold: {season: [list of column names]}
season_columns = {}

all_dfs = []

for file in csv_files:
    # Build Season string like '2000/01'
    raw = file.replace('.csv', '')
    start = 2000 + int(raw[:2])
    end   = 2000 + int(raw[2:])
    season = f"{start}/{str(end)[-2:]}"
    
    path = os.path.join(data_folder, file)
    try:
        df = pd.read_csv(path, on_bad_lines='skip', encoding='latin1')
        
        # Drop first column (e.g., 'Div') by position
        df = df.iloc[:, 1:]

        # Drop 'Time' and 'Referee' if present
        for col_to_drop in ['Time', 'Referee']:
            if col_to_drop in df.columns:
                df.drop(columns=col_to_drop, inplace=True)
        
        # Add Season column
        df['Season'] = season
        all_dfs.append(df)
        print(f"✅ Processed {file} → Season: {season}, shape: {df.shape}")

    except Exception as e:
        print(f"❌ Error loading {file}: {e}")

# Combine all into one DataFrame
epl_combined = pd.concat(all_dfs, ignore_index=True)
print(f"\n✅ Combined dataset shape: {epl_combined.shape}")

✅ Processed 0001.csv → Season: 2000/01, shape: (380, 44)
✅ Processed 0102.csv → Season: 2001/02, shape: (380, 47)
✅ Processed 0203.csv → Season: 2002/03, shape: (380, 52)
✅ Processed 0304.csv → Season: 2003/04, shape: (335, 56)
✅ Processed 0405.csv → Season: 2004/05, shape: (335, 56)
✅ Processed 0506.csv → Season: 2005/06, shape: (380, 67)
✅ Processed 0607.csv → Season: 2006/07, shape: (380, 67)
✅ Processed 0708.csv → Season: 2007/08, shape: (380, 70)
✅ Processed 0809.csv → Season: 2008/09, shape: (380, 70)
✅ Processed 0910.csv → Season: 2009/10, shape: (380, 70)
✅ Processed 1011.csv → Season: 2010/11, shape: (380, 70)
✅ Processed 1112.csv → Season: 2011/12, shape: (380, 70)
✅ Processed 1213.csv → Season: 2012/13, shape: (380, 73)
✅ Processed 1314.csv → Season: 2013/14, shape: (380, 67)
✅ Processed 1415.csv → Season: 2014/15, shape: (381, 67)
✅ Processed 1516.csv → Season: 2015/16, shape: (380, 64)
✅ Processed 1617.csv → Season: 2016/17, shape: (380, 64)
✅ Processed 1718.csv → Season: 

In [3]:
epl_combined.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Attendance,...,1XBCH,1XBCD,1XBCA,BFECH,BFECD,BFECA,BFEC>2.5,BFEC<2.5,BFECAHH,BFECAHA
0,19/08/00,Charlton,Man City,4.0,0.0,H,2.0,0.0,H,20043.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19/08/00,Chelsea,West Ham,4.0,2.0,H,1.0,0.0,H,34914.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19/08/00,Coventry,Middlesbrough,1.0,3.0,A,1.0,1.0,D,20624.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19/08/00,Derby,Southampton,2.0,2.0,D,1.0,2.0,A,27223.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19/08/00,Leeds,Everton,2.0,0.0,H,2.0,0.0,H,40010.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,19/08/00,Leicester,Aston Villa,0.0,0.0,D,0.0,0.0,D,21455.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,19/08/00,Liverpool,Bradford,1.0,0.0,H,0.0,0.0,D,44183.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,19/08/00,Sunderland,Arsenal,1.0,0.0,H,0.0,0.0,D,46346.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,19/08/00,Tottenham,Ipswich,3.0,1.0,H,2.0,1.0,H,36148.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,20/08/00,Man United,Newcastle,2.0,0.0,H,1.0,0.0,H,67477.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleaning 

In [4]:
epl_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9381 entries, 0 to 9380
Columns: 190 entries, Date to BFECAHA
dtypes: float64(184), object(6)
memory usage: 13.6+ MB


### Drop betting stats and Keep only match stats

In [5]:
columns_to_keep = [
    'Date', 'HomeTeam', 'AwayTeam',
    'FTHG', 'HG', 'FTAG', 'AG', 'FTR', 'Res',
    'HTHG', 'HTAG', 'HTR', 'Attendance',
    'HS', 'AS', 'HST', 'AST', 'HHW', 'AHW',
    'HC', 'AC', 'HF', 'AF', 'HFKC', 'AFKC',
    'HO', 'AO', 'HY', 'AY', 'HR', 'AR',
]

# Ensure all columns exist before filtering
filtered_columns = [col for col in columns_to_keep if col in epl_combined.columns]

# Place 'Season' at the front
final_columns = ['Season'] + filtered_columns

# Filter and reorder
epl_final = epl_combined[final_columns]

display(epl_final)

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HC,AC,HF,AF,HO,AO,HY,AY,HR,AR
0,2000/01,19/08/00,Charlton,Man City,4.0,0.0,H,2.0,0.0,H,...,6.0,6.0,13.0,12.0,8.0,6.0,1.0,2.0,0.0,0.0
1,2000/01,19/08/00,Chelsea,West Ham,4.0,2.0,H,1.0,0.0,H,...,7.0,7.0,19.0,14.0,2.0,3.0,1.0,2.0,0.0,0.0
2,2000/01,19/08/00,Coventry,Middlesbrough,1.0,3.0,A,1.0,1.0,D,...,8.0,4.0,15.0,21.0,1.0,3.0,5.0,3.0,1.0,0.0
3,2000/01,19/08/00,Derby,Southampton,2.0,2.0,D,1.0,2.0,A,...,5.0,8.0,11.0,13.0,0.0,2.0,1.0,1.0,0.0,0.0
4,2000/01,19/08/00,Leeds,Everton,2.0,0.0,H,2.0,0.0,H,...,6.0,4.0,21.0,20.0,6.0,1.0,1.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9376,2024/25,04/05/2025,Brentford,Man United,4.0,3.0,H,2.0,1.0,H,...,7.0,4.0,8.0,10.0,NaN,NaN,0.0,2.0,0.0,0.0
9377,2024/25,04/05/2025,Brighton,Newcastle,1.0,1.0,D,1.0,0.0,H,...,1.0,4.0,15.0,10.0,NaN,NaN,2.0,1.0,0.0,0.0
9378,2024/25,04/05/2025,West Ham,Tottenham,1.0,1.0,D,1.0,1.0,D,...,1.0,3.0,18.0,15.0,NaN,NaN,2.0,2.0,0.0,0.0
9379,2024/25,04/05/2025,Chelsea,Liverpool,3.0,1.0,H,1.0,0.0,H,...,3.0,6.0,10.0,11.0,NaN,NaN,2.0,2.0,0.0,0.0


### Handle Missing Value

In [6]:
missing_counts = epl_final.isnull().sum()
missing_percent = epl_final.isnull().mean() * 100

missing_summary = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing %': missing_percent.round(2)
}).sort_values(by='Missing Count', ascending=False)

display(missing_summary)

,Missing Count,Missing %
Attendance,8622,91.91
HHW,8621,91.90
AHW,8621,91.90
AO,8621,91.90
HO,8621,91.90
FTAG,1,0.01
FTHG,1,0.01
FTR,1,0.01
AwayTeam,1,0.01
HomeTeam,1,0.01


'Attendance', 'HHW', 'AHW', 'AO' and 'HO' contain a high proportion of missing value. Therefore we are dropping them.

In [8]:
columns_to_drop = ['Attendance', 'HHW', 'AHW', 'HO', 'AO']

epl_final = epl_final.drop(columns=[col for col in columns_to_drop if col in epl_final.columns])

In [9]:
rows_with_missing = epl_final[epl_final.isnull().any(axis=1)]

rows_with_missing.head()

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
5610,2014/15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Drop rows with any missing values
epl_final = epl_final.dropna()

## Rename Columns

In [11]:
epl_final.head()

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
0,2000/01,19/08/00,Charlton,Man City,4.0,0.0,H,2.0,0.0,H,...,14.0,4.0,6.0,6.0,13.0,12.0,1.0,2.0,0.0,0.0
1,2000/01,19/08/00,Chelsea,West Ham,4.0,2.0,H,1.0,0.0,H,...,10.0,5.0,7.0,7.0,19.0,14.0,1.0,2.0,0.0,0.0
2,2000/01,19/08/00,Coventry,Middlesbrough,1.0,3.0,A,1.0,1.0,D,...,3.0,9.0,8.0,4.0,15.0,21.0,5.0,3.0,1.0,0.0
3,2000/01,19/08/00,Derby,Southampton,2.0,2.0,D,1.0,2.0,A,...,4.0,6.0,5.0,8.0,11.0,13.0,1.0,1.0,0.0,0.0
4,2000/01,19/08/00,Leeds,Everton,2.0,0.0,H,2.0,0.0,H,...,8.0,6.0,6.0,4.0,21.0,20.0,1.0,3.0,0.0,0.0


In [12]:
rename_map = {
    'Date': 'MatchDate',
    'FTHG': 'FullTimeHomeGoals',
    'FTAG': 'FullTimeAwayGoals',
    'FTR': 'FullTimeResult',
    'HTHG': 'HalfTimeHomeGoals',
    'HTAG': 'HalfTimeAwayGoals',
    'HTR': 'HalfTimeResult',
    'HS': 'HomeShots',
    'AS': 'AwayShots',
    'HST': 'HomeShotsOnTarget',
    'AST': 'AwayShotsOnTarget',
    'HC': 'HomeCorners',
    'AC': 'AwayCorners',
    'HF': 'HomeFouls',
    'AF': 'AwayFouls',
    'HY': 'HomeYellowCards',
    'AY': 'AwayYellowCards',
    'HR': 'HomeRedCards',
    'AR': 'AwayRedCards'
}

epl_final = epl_final.rename(columns=rename_map)

## Check Data Types and Reindex

In [14]:
print(epl_final.dtypes)

Season                object
MatchDate             object
HomeTeam              object
AwayTeam              object
FullTimeHomeGoals    float64
FullTimeAwayGoals    float64
FullTimeResult        object
HalfTimeHomeGoals    float64
HalfTimeAwayGoals    float64
HalfTimeResult        object
HomeShots            float64
AwayShots            float64
HomeShotsOnTarget    float64
AwayShotsOnTarget    float64
HomeCorners          float64
AwayCorners          float64
HomeFouls            float64
AwayFouls            float64
HomeYellowCards      float64
AwayYellowCards      float64
HomeRedCards         float64
AwayRedCards         float64
dtype: object


In [ ]:
# Covert to datetime
epl_final['MatchDate'] = pd.to_datetime(epl_final['MatchDate'], dayfirst=True)

In [ ]:
# Convert goal columns to integers
goal_cols = [
    'FullTimeHomeGoals', 'FullTimeAwayGoals', 'HalfTimeHomeGoals', 'HalfTimeAwayGoals',
    'HomeShots', 'AwayShots', 'HomeShotsOnTarget', 'AwayShotsOnTarget',
    'HomeCorners', 'AwayCorners', 'HomeFouls', 'AwayFouls',
    'HomeYellowCards', 'AwayYellowCards', 'HomeRedCards', 'AwayRedCards'
]

for col in goal_cols:
    if col in epl_final.columns:
        epl_final[col] = pd.to_numeric(epl_final[col], errors='coerce').astype('Int64')

In [21]:
epl_final.reset_index(drop=True, inplace=True)

In [22]:
print(epl_final.dtypes)

Season                       object
MatchDate            datetime64[ns]
HomeTeam                     object
AwayTeam                     object
FullTimeHomeGoals             Int64
FullTimeAwayGoals             Int64
FullTimeResult               object
HalfTimeHomeGoals             Int64
HalfTimeAwayGoals             Int64
HalfTimeResult               object
HomeShots                     Int64
AwayShots                     Int64
HomeShotsOnTarget             Int64
AwayShotsOnTarget             Int64
HomeCorners                   Int64
AwayCorners                   Int64
HomeFouls                     Int64
AwayFouls                     Int64
HomeYellowCards               Int64
AwayYellowCards               Int64
HomeRedCards                  Int64
AwayRedCards                  Int64
dtype: object


## Export csv file

In [ ]:
# epl_final.to_csv('../data/processed/epl_final.csv', index=False)